In [21]:
import numpy #use for arrays
from scipy import linalg #for solving linear system
from matplotlib import pyplot
%matplotlib inline

In [22]:
L = 1.0
nx = 51
dx = L / (nx-1)
x = numpy.linspace(0.0, L, num=nx)

alpha = 1.22e-3

T0 = numpy.zeros(nx)
T0[0] = 100.0

In [106]:
#This code will be more generic forward, backward, Crank Nicholson kullanacagiz, boundary condition lara sonradan karar verecegiz
def solve_1d(T0, nt, dt, dx, alpha, cI, cE, bc):
    T = T0.copy() #making deep copy of the initial array
    N = len(T0) - 2 #number of variables inside my domain
    A, b_bc = lhs_operator(N, dt, dx, alpha, cE) #left hand side operator
    for n in range(nt):
        b = rhs_vector(T, dt, dx, alpha, cE)  #Every time step I redefine my right hand side vector
        T[1:-1] = linalg.solve(A,b-b_bc) #right hand side i bir tarafa topladi
        update_boundaries(T, dx, bc)
    return T
        

In [107]:
def lhs_operator(N, dt, dx, alpha, cI, bc): #Matrix A
    I = numpy.identity(N) #diagonal matrix with only 1 on the diagonal
    L, b_bc = laplacian_1d(N, dx, bc) #Laplacian operator
    A = I / dt - cI * alpha * L #I an L are matrices so A is also going to be matrix    
    return A, b_bc

In [116]:
def laplacian_1d(N, dx,bc):
    D = numpy.diag(-2.0 / dx**2 * numpy.ones(N)) #inputs should be 1d array
    L = numpy.diag(1.0 / dx**2 * numpy.ones(N-1), k=-1) #I want to put them in the lower diagonal
    U = numpy.diag(1.0 / dx**2 * numpy.ones(N-1), k =+1)#Upper diagonal
    b_bc = numpy.zeros(N)
    A = D + U + L
    #Left boundary #correction for the left boundary and right boundary so we will use if conditions for that
    if bc['left']['type'] == 'Dirichlet': #type and value hakkinda bilgi verecegim NEW INFO
        b_bc[0] = bc['left']['value'] / dx**2
    elif bc['left']['type'] == 'Neumann':
        A[0, 0] = - 1.0 / dx**2
        b_bc[0] = bc['left']['value'] / dx
    #Right boundary
    if bc['right']['type'] == 'Dirichlet': 
        b_bc[-1] = bc['right']['value'] / dx**2
    elif bc['right']['type'] == 'Neumann':
        A[-1, -1] = - 1.0 / dx**2
        b_bc[-1] = bc['right']['value'] / dx
    
    
    return A, b_bc

In [117]:
#dictionary contains item and value
#we want to know type of boundary condition and the value of the boundary
bc_left = {'type' : 'Dirichlet','value': 100.0} #what we want to impose on left boundary 100.0
bc_right = {'type' : 'Neumann','value': 0.0} 
print(bc_left['type'])
print(bc_left['value'])
bc = {'left': bc_left, 'right': bc_right}
print(bc_left['type'])
print(bc['left']['type'])

Dirichlet
100.0
Dirichlet
Dirichlet


In [118]:
def rhs_vector(T, dt, dx, alpha, cE, bc):
    b = T[1:-1] / dt + cE*alpha* (T[:-2] -2*T[1:-1] +T[2:]) / dx**2  #HATA OLABILIR
    return b

In [119]:
def update_boundaries(T, dx, bc):
    T[0] = get_boundary_value(T[1], dx, bc['left'])
    T[-1] = get_boundary_value(T[-2], dx, bc['right'])
    return

In [120]:
def get_boundary_value(T_neighbor, dx, bc): #I need access to Ti
    if bc['type'] == 'Dirichlet':
        return bc['value']
    elif bc['type'] == 'Neumann':
        return T_neighbor + bc['value']*dx
    else:
        raise ValueError('Only Dirichlet and Neumann are supported')
    return
    

In [121]:
Tb = get_boundary_value(100.0, 1.0, {'type': 'Dirichlet', 'value': 101.0})
Tb

101.0

In [122]:
#Fake example to show modification in place (deep copy yapmaktan ve memory overload etmekten kurtariyor)
def func(a):
    a[0] += 2.0
    return
a = numpy.ones(3)
print(a)
func(a)
print(a)

[1. 1. 1.]
[3. 1. 1.]


In [123]:
#We will define dt based on sigma
sigma = 0.5
dt = sigma*dx**2/alpha
nt = 100

T = solve_1d(T0, nt, dt, dx, alpha, 0.5, 0.5, bc)

TypeError: lhs_operator() missing 1 required positional argument: 'bc'

In [87]:
pyplot.plot

<function matplotlib.pyplot.plot>